<a href="https://colab.research.google.com/github/Klim2007/wall_mart_sales_kaggle/blob/main/WallMart13_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PREDICTION OF SALES BY STORES, CPI & FUEL PRICE 
(USING XGBOOST XGREGRESSOR)

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [81]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib as plt
from datetime import datetime as dt
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from xgboost import XGBRegressor
import plotly.express as px
import plotly.graph_objects as go
import datetime

DATA PREPROCESSING

In [83]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/walmart-sales-dataset-of-45stores.csv',parse_dates=['Date'])

In [84]:
df

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,2010-05-02,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,2010-12-02,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,2010-05-03,1554806.68,0,46.50,2.625,211.350143,8.106
...,...,...,...,...,...,...,...,...
6430,45,2012-09-28,713173.95,0,64.88,3.997,192.013558,8.684
6431,45,2012-05-10,733455.07,0,64.89,3.985,192.170412,8.667
6432,45,2012-12-10,734464.36,0,54.47,4.000,192.327265,8.667
6433,45,2012-10-19,718125.53,0,56.47,3.969,192.330854,8.667


In [85]:
training_data = df.sample(frac=0.8, random_state=25)
testing_data = df.drop(training_data.index)

In [44]:
training_data.describe()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,5148.000000,5.148000e+03,5148.000000,5148.000000,5148.000000,5148.000000,5148.000000
mean,22.942113,1.050277e+06,0.070707,60.708261,3.357503,171.758562,8.021854
std,13.000506,5.675674e+05,0.256360,18.342991,0.459921,39.369420,1.895840
min,1.000000,2.099862e+05,0.000000,-2.060000,2.472000,126.064000,3.879000
25%,12.000000,5.540791e+05,0.000000,47.660000,2.923000,131.624403,6.901000
50%,23.000000,9.636464e+05,0.000000,62.635000,3.452000,182.833244,7.896000
75%,34.000000,1.424810e+06,0.000000,74.832500,3.734000,212.760898,8.601750
max,45.000000,3.818686e+06,1.000000,100.140000,4.468000,227.232807,14.313000


In [45]:
training_data.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
5800,41,2011-08-19,1412959.97,0,70.55,3.499,194.250063,6.901
5435,39,2010-12-02,1266229.07,1,44.58,2.548,209.997021,8.554
2304,17,2010-05-28,872817.62,0,48.19,2.908,126.160226,6.635
824,6,2012-09-03,1569304.40,0,57.89,3.669,222.665100,6.132
4512,32,2011-12-08,1206795.74,0,74.20,3.542,194.110502,8.622


In [46]:
training_data.dtypes

Store                    int64
Date            datetime64[ns]
Weekly_Sales           float64
Holiday_Flag             int64
Temperature            float64
Fuel_Price             float64
CPI                    float64
Unemployment           float64
dtype: object

In [47]:
training_data.value_counts()

Store  Date        Weekly_Sales  Holiday_Flag  Temperature  Fuel_Price  CPI         Unemployment
1      2010-01-10  1453329.50    0             71.89        2.603       211.671989  7.838           1
30     2011-06-05  438789.52     0             61.87        3.906       215.444871  7.931           1
       2012-10-19  437537.29     0             68.52        3.594       223.059808  6.170           1
       2012-10-08  430878.28     0             89.57        3.494       221.595414  6.565           1
       2012-09-28  425410.04     0             80.38        3.666       222.616433  6.565           1
                                                                                                   ..
15     2012-05-25  693780.42     0             67.97        3.979       138.110194  8.150           1
       2012-05-10  573498.64     0             59.57        4.151       138.825600  7.992           1
       2012-04-27  527402.62     0             41.57        4.163       137.978133  8.1

Analyzing data and dependancies
1. Store Sales Hollidays vs Working days

In [48]:
store_sales_working = training_data[training_data['Holiday_Flag'].isin([0])]
sales_stores = store_sales_working.groupby('Store').Weekly_Sales.mean().reset_index()
fig = px.bar(sales_stores, x = "Store", y ="Weekly_Sales",color="Store",title="Store weekly Sales",height=400)
fig.show()

2.Hollyday_Sales

In [49]:
store_sales_hollidays = training_data[training_data['Holiday_Flag'].isin([1])]
a = store_sales_hollidays.groupby(by="Store", dropna=False).mean().reset_index()
fig = px.bar(a, x='Store', y='Weekly_Sales',color='Store',title="Stores Overall Sales")
fig.show()
     

employment working days vs hollydays

In [86]:
store_emp_hollidays = training_data[training_data['Holiday_Flag'].isin([1])]
store_emp_wrk = training_data[training_data['Holiday_Flag'].isin([0])]

store_emp_hollidays['mounth'] = store_emp_hollidays['Date'].dt.month
store_emp_wrk['mounth'] = store_emp_wrk['Date'].dt.month

<ipython-input-86-710d9aa0f5ca>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-86-710d9aa0f5ca>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [51]:
store_emp_hollidays = store_emp_hollidays.groupby('mounth')['Unemployment'].mean()
store_emp_wrk = store_emp_wrk.groupby('mounth')['Unemployment'].mean()

In [52]:
new_index = []
for i in range (1,13): new_index.append(i)
store_emp_hol = store_emp_hollidays.reindex(new_index)

In [53]:
store_emp_hol

mounth
1          NaN
2          NaN
3          NaN
4          NaN
5          NaN
6          NaN
7     7.212868
8          NaN
9     8.307765
10    8.033903
11    8.335658
12    8.371128
Name: Unemployment, dtype: float64

In [54]:
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun','Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=months,
    y= store_emp_hol,
    name='unemployment on holidays',
    marker_color='indianred'
))
fig.add_trace(go.Bar(
    x=months,
    y= store_emp_wrk,
    name='unemployment on working days',
    marker_color='lightsalmon'
))

We see that unemployment on holidays is much higher 

CPI index review

In [55]:
CPI_stores = training_data.groupby('Store').CPI.mean().reset_index()
fig = px.bar(CPI_stores, x='Store', y='CPI',color='Store',title="CPI by Stores")
fig.update_layout(barmode='stack', xaxis={'categoryorder':'category ascending'})
fig.show()

In [56]:
CPI_hollidays = training_data[training_data['Holiday_Flag'].isin([1])]
CPI_wrk = training_data[training_data['Holiday_Flag'].isin([0])]

CPI_hollidays['mounth'] = CPI_hollidays['Date'].dt.month
CPI_wrk['mounth'] = CPI_wrk['Date'].dt.month

<ipython-input-56-d47ea4ce0427>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-56-d47ea4ce0427>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [57]:
CPI_hollidays.columns

Index(['Store', 'Date', 'Weekly_Sales', 'Holiday_Flag', 'Temperature',
       'Fuel_Price', 'CPI', 'Unemployment', 'mounth'],
      dtype='object')

In [58]:
CPI_on_hollidays = CPI_hollidays.groupby('mounth')['CPI'].mean()
CPI_on_wrk = CPI_wrk.groupby('mounth')['CPI'].mean()

In [59]:
new_index = []
for i in range (1,13): new_index.append(i)
CPI_on_hollidays = CPI_on_hollidays.reindex(new_index)

In [60]:
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun','Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=months,
    y= CPI_on_hollidays,
    name='CPI on holidays',
    marker_color='indianred'
))
fig.add_trace(go.Bar(
    x=months,
    y= CPI_on_wrk,
    name='CPI on working days',
    marker_color='lightsalmon'
))

In [77]:
def split_year(time):
    return (time.split('-')[0])
def split_month(time):
    return (time.split('-')[1])
def split_day(time):
    return (time.split('-')[2])


In [75]:
training_data['Date']

5800    2011-08-19 00:00:00
5435    2010-12-02 00:00:00
2304    2010-05-28 00:00:00
824     2012-09-03 00:00:00
4512    2011-12-08 00:00:00
               ...         
436     2010-03-26 00:00:00
4976    2012-04-13 00:00:00
5765    2010-12-17 00:00:00
1837    2012-01-06 00:00:00
6286    2012-09-21 00:00:00
Name: Date, Length: 5148, dtype: object

In [87]:
training_data['Year'] = training_data['Date'].dt.strftime('%Y')
#training_data['Month'] = training_data['Date'].dt.mounth
#training_data['Day'] = training_data['Date'].apply(split_day)

In [88]:
training_data

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year
5800,41,2011-08-19,1412959.97,0,70.55,3.499,194.250063,6.901,2011
5435,39,2010-12-02,1266229.07,1,44.58,2.548,209.997021,8.554,2010
2304,17,2010-05-28,872817.62,0,48.19,2.908,126.160226,6.635,2010
824,6,2012-09-03,1569304.40,0,57.89,3.669,222.665100,6.132,2012
4512,32,2011-12-08,1206795.74,0,74.20,3.542,194.110502,8.622,2011
...,...,...,...,...,...,...,...,...,...
436,4,2010-03-26,1762539.30,0,39.91,2.752,126.605064,8.623,2010
4976,35,2012-04-13,788633.42,0,52.22,4.044,141.843393,8.876,2012
5765,41,2010-12-17,1627904.68,0,31.51,2.778,191.030338,7.508,2010
1837,13,2012-01-06,2035431.39,0,61.11,3.788,130.959226,5.965,2012
